In [ ]:
# CHEM 274B Final Project: Cellular Automata Library
# Shiqi Zhang, Jerry Pan, Wang Xu
# December 2022

# Directory Path: Utils/Data/Chem274B-CAModels.ipynb
# This file contains the output visualization of the
# input setup phase for the cellular automata library.

# Simple CA simulator in Python
#
# *** Cancer Metastasis ***

import matplotlib
import matplotlib.pyplot as plt

# import the python libraries random (RD), SciPy (SP)
import random as RD
import scipy as SP
import pandas as pd

RD.seed()

In [ ]:
# First we process the file with matrices of cell status, represented by pre-defined integers;

# Open the data file;
dfile = open('xxx.txt','r') 

# First line of the file has the metadata;
nt =int(dfile.readline())
print(nt)

# Additional metadata;
"""width = 100
height = 100
scenario = 1
initProb = 0.5"""

# Non-cancer cell is represented by 0, dark green
# Benevolent cancer cell is represented by 1, light green
# Malignant cancer cell is represented by 2, yellow
# Apoptotic cell is represented by 3, red
non, bene, mali, apop = range(4)

# Set Color map
cmap_reversed = matplotlib.cm.get_cmap('RdYlGn_r')

# Initialize a dataframe to store all data
df = pd.DataFrame(columns=['Time', 'Non-Cancerous Cells', 'Benevolent Cells', 'Maliganent Cells', 'Apoptotic Cells'])

In [ ]:
# Define functions to initialize, progress and draw the cell map

# Function Initialize (or Setup)
# Defines the cellular automata (size and initial states)
def init():
    global time, config, nextConfig

    time = 0
    
    # Use metadata from read file to define matrix map size
    config = SP.zeros([height, width])

    # Set initial cell status to non-cancerous or benevolent cells
    for x in range(width):
        for y in range(height):
            if RD.random() < initProb:
                state = non
            else:
                state = bene
            config[y, x] = state
    
    # Set a maliganent cell at the middle
    config[int(height/2), int(width/2)] = mali

    # Return percentage of different type cells from the initial status
    non_ct = 0
    bene_ct = 0
    mali_ct = 0
    apop_ct = 0
    total = width * height
    for x in range(width):
        for y in range(height):
            if state == non:
                non_ct += 1
            elif state == bene:
                bene_ct += 1
            elif state == mali:
                mali_ct += 1
            elif state == apop:
                apop_ct += 1
    assert non_ct + bene_ct + mali_ct + apop_ct == total, "Number of cells incorrect"
    non_pct = round(non_ct / total * 100, 1)
    bene_pct = round(bene_ct / total * 100, 1)
    mali_pct = round(mali_ct / total * 100, 1)
    apop_pct = round(apop_ct / total * 100, 1)

    nextConfig = SP.zeros([height, width])
    new_row = [time, non_pct, bene_pct, mali_pct, apop_pct]
    df.loc[time] = new_row
    print(f"non-cancerous, benevolent, maliganent, apoptotic cell percent are: {non_pct}%, {bene_pct}%, {mali_pct}%, {apop_pct}%.")

In [ ]:
# Function to define each step of cell mapping
# Defines the cellular automata (current status and next-timeframe status)
def step():
    global time, config, nextConfig

    time += 1

    for x in range(width):
        for y in range(height):
            state = config[y, x]
            if state == mali:
                # the maliganent cell goes into apoptosis after one time step
                state = apop
            elif state == bene:
                # the benevalent cell next to maliganent cell goes rogue after one time step
                for dx in range(-1, 2):
                    for dy in range(-1, 2):
                        if config[(y+dy)%height, (x+dx)%width] == mali:
                            state = mali
            elif state == non:
                # the non-cancerous cell next to maliganent cell has chance to turn into 
                # benevolent cancer cell after one time step
                for dx in range(-1, 2):
                    for dy in range(-1, 2):
                        if config[(y+dy)%height, (x+dx)%width] == mali:
                            if RD.random() > initProb:
                                state = bene
            nextConfig[y, x] = state

    # Return percentage of different type cells from each step
    non_ct = 0
    bene_ct = 0
    mali_ct = 0
    apop_ct = 0
    total = width * height
    for x in range(width):
        for y in range(height):
            if state == non:
                non_ct += 1
            elif state == bene:
                bene_ct += 1
            elif state == mali:
                mali_ct += 1
            elif state == apop:
                apop_ct += 1
    assert non_ct + bene_ct + mali_ct + apop_ct == total, "Number of cells incorrect"
    non_pct = round(non_ct / total * 100, 1)
    bene_pct = round(bene_ct / total * 100, 1)
    mali_pct = round(mali_ct / total * 100, 1)
    apop_pct = round(apop_ct / total * 100, 1)

    config, nextConfig = nextConfig, config
    new_row = [time, non_pct, bene_pct, mali_pct, apop_pct]
    df.loc[time] = new_row
    print(f"non-cancerous, benevolent, maliganent, apoptotic cell percent are: {non_pct}%, {bene_pct}%, {mali_pct}%, {apop_pct}%.")

In [ ]:
# Define the function to 
def draw():
    plt.cla()
    global time, config, nextConfig
    plt.pcolor(config, vmin = 0, vmax = 4, cmap = cmap_reversed, label = config)
    plt.axis('image')
    plt.title('initProb = ' + str(initProb) + ' ,  ' + 't = ' + str(time))
    plt.show()

In [ ]:
# Implement the cell mapping tool generated from cellular automata data
init()
draw()
plt.figure()
for i in range(100):
    step()
    draw()
    plt.figure()

# Write pandas into file;
dfile1 = open('xxx.csv','w') 
df.to_csv('xxx.csv', index=False)